# BPNN training and testing 

In [2]:
import torch
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import argparse
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader
import random
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


import torch.optim as optim
import torchvision
from torchvision import datasets,transforms 

from skorch import NeuralNetClassifier

from sklearn.model_selection import train_test_split
import Model
from trainer import Trainer
import dataloader

Starting by creating options for training, testing and usage.

In [3]:
class Args:
    label_dir = "./Label_5p.csv"
    image_dir = "./data/ROI_trab"
    train_cross = "./cross_output.pkl"
    batch_size = 8
    model = "ConvNet" 
    nof = 8
    lr= 0.001
    nb_epochs = 5
    checkpoint_path= "./"
    mode= "Train"
    cross_val = False
    k_fold= 5
    n1= 240
    n2= 120
    n3 = 60
    nb_workers = 0
    norm_method="standardization"

opt = Args()
NB_DATA = 3991
NB_LABEL = 5
PERCENTAGE_TEST = 20
RESIZE_IMAGE = 512

Let's check if gpu is available 

In [4]:
if torch.cuda.is_available():  
  device = "cuda:0"
  print("running on gpu")
else:  
  device = "cpu"
  print("running on cpu")

running on cpu


We start by importing the Network. Here we choose *ConvNet* but you can also choose *ResNet50*, *ResNet101* and *Unet*. 

In [7]:
model = Model.ConvNet(features =opt.nof,out_channels=NB_LABEL,k1 = 3,k2 = 3,k3= 3).to(device)

In [5]:
net = NeuralNetClassifier(
    Model.ConvNet(features = opt.nof,out_channels=NB_LABEL),
    max_epochs = 10,
    iterator_train__num_workers=0,
    iterator_valid__num_workers=0,
    lr=0.001,
    batch_size=8,
    optimizer=optim.Adam,
    criterion=torch.nn.MSELoss
)

We prepare the splitting of the dataset, and we normalize parameters label.

In [6]:
index = range(NB_DATA)
split = train_test_split(index,test_size = 0.2,random_state=1)

In [7]:
print(split[0])

[3247, 3729, 2707, 2425, 3473, 902, 1448, 990, 3108, 1865, 2698, 1373, 1380, 1176, 1379, 305, 2374, 439, 713, 3333, 953, 1836, 1271, 3045, 2164, 2930, 58, 1475, 664, 1710, 1962, 650, 2978, 462, 2765, 50, 772, 2098, 1078, 306, 1759, 2424, 2241, 65, 1661, 3233, 870, 2289, 3767, 195, 3226, 3970, 1167, 2997, 3119, 3900, 1744, 3369, 2045, 1847, 1625, 2000, 2206, 2259, 2697, 396, 1595, 1856, 3719, 2606, 1776, 586, 3803, 957, 1006, 2299, 473, 2733, 3482, 3483, 3937, 2039, 2238, 2833, 2814, 1459, 3246, 407, 137, 797, 1140, 1417, 3420, 3403, 1047, 1903, 1398, 2682, 834, 516, 3071, 955, 2950, 220, 2998, 3261, 3596, 3070, 2183, 2608, 1221, 3279, 1649, 725, 1503, 1642, 282, 3466, 1918, 1065, 3366, 1404, 2907, 2213, 3096, 471, 1038, 1501, 1173, 1162, 11, 3598, 415, 988, 1906, 2135, 1546, 1179, 2117, 757, 2749, 2719, 1807, 544, 3173, 2520, 3661, 2847, 2323, 1248, 3311, 1782, 671, 945, 223, 613, 3257, 3683, 1338, 2469, 2688, 2726, 2328, 180, 3856, 1030, 125, 2142, 730, 616, 812, 2856, 2935, 111, 3065

In [8]:
scaler = dataloader.normalization(opt.label_dir,opt.norm_method,split[0])

Dataset creation from label and images

In [13]:
datasets = dataloader.Datasets(csv_file = opt.label_dir, image_dir = opt.image_dir, opt=opt, indices =split[0]) # Create dataset
trainloader = DataLoader(datasets, batch_size = opt.batch_size,  num_workers = opt.nb_workers )

In [47]:
y_train = np.array([y for x, y in iter(trainloader)])
x_train = np.array([x for x, y in iter(trainloader)])

In [55]:
print(x_train[2])

image


In [1]:
y_train = np.array([data['label'] for i,data in enumerate(datasets)])


NameError: name 'np' is not defined

Here, let's create our cross validation dataset

In [3]:
net.fit(trainloader,y=y_train)

NameError: name 'net' is not defined